# CAI Lab Session 2: Intro to ElasticSearch

In this session you will learn:

- a few basics of the `ElasticSearch` database
- how to index a set of documents and how to ask simple queries about these documents
- how to do this from `Python`
- based on the previous, you will compute the boolean and tf-idf matrix for the toy corpus used in class

## 1. ElasticSearch

[ElasticSearch](https://www.elastic.co/) is a _NoSQL/document_ database with the capability of indexing and searching text documents. As a rough analogue, we can use the following table for the equivalence between ElasticSearch and a more classical relational database:

| Relational DB | ElasticSearch |
|---|---|
| Database | Index |
| Table | Type |
| Row / record | Document |
| Column | Field |

An index can be thought of as an optimized collection of documents and each document is a collection of fields, which are the key-value pairs that contain your data.

`ElasticSearch` is a pretty big beast with many options. Luckily, there is much documentation, a few useful links are:

- Here is the [full documentation](https://www.elastic.co/guide/en/elasticsearch/reference/current/index.html)
- Intros you may want to have a look at: 
    - https://medium.com/expedia-group-tech/getting-started-with-elastic-search-6af62d7df8dd
    - http://joelabrahamsson.com/elasticsearch-101
- You found another one that you liked? Let us know. 

## 2. Running ElasticSearch

First you will need to install `ElasticSearch` following instructions in their [documentation](https://www.elastic.co/guide/en/elasticsearch/reference/current/install-elasticsearch.html).

This database runs as a web service in a machine and can be accessed using a REST web API; however we will interact with the database through its python libraries `elasticsearch-py` and `elasticsearch-dsl`, so you will need to install these as well.  You can run `ElasticSearch` by typing from the command-line prompt:

```
$ <path_to_elasticsearch_bin>/elasticsearch &
```



After a few seconds (and a lot of logging) the database will be up and running; you may need to hit return for the prompt to show up. To test whether `ElasticSearch` is working execute the code in the cell below. __The database needs to be running throughout the execution of this script, otherwise you will get a connection error.__

In [1]:
import requests
import numpy as np
from pprint import pprint

In [2]:
try:
    resp = requests.get('http://localhost:9200/')
    pprint(resp.content)
    
except Exception:
    print('elasticsearch is not running')

(b'{\n  "name" : "DESKTOP-QNOM0EQ",\n  "cluster_name" : "elasticsearch",\n  "c'
 b'luster_uuid" : "eig_fG5ySRWEscM_Y6LUcQ",\n  "version" : {\n    "number" : '
 b'"8.10.2",\n    "build_flavor" : "default",\n    "build_type" : "zip",\n    '
 b'"build_hash" : "6d20dd8ce62365be9b1aca96427de4622e970e9e",\n    "build_da'
 b'te" : "2023-09-19T08:16:24.564900370Z",\n    "build_snapshot" : false,\n  '
 b'  "lucene_version" : "9.7.0",\n    "minimum_wire_compatibility_version" :'
 b' "7.17.0",\n    "minimum_index_compatibility_version" : "7.0.0"\n  },\n  "t'
 b'agline" : "You Know, for Search"\n}\n')


If `ElasticSearch` is working you will see an answer from the server; otherwise you will see a message indicating that it is not running. You can try also throwing the URL http://localhost:9200 to your browser; you should get a similar answer.

**In version 8 they introduced enhanced security, which may give you trouble when executing the code here, to deal with this you can either install an earlier version (7 or older) or turn off security settings in their `config/elasticsearch.yml` config file (just set to _false_ everything concerning the security options).** Since we are using the database in offline, local mode this should not be a problem.

Also, you should run this script locally in your machine, if you use Google Collab or similar this is not going to work because elasticsearch should be running on the machine where the script is being executed.

## 3. Indexing and querying

`ElasticSearch` is a database that allows storing documents (tables do not need a predefined schema as in relational databases). Text in these documents can be processed so the queries extend beyond exact matches allowing complex queries, fuzzy matching and ranking documents respect to the actual match. 

These kinds of databases are behind search engines like Google Search or Bing.

There are different ways of operating with ElasticSearch. It is deployed esentially as a web service with a REST API, so it can be accessed basically from any language with a library for operating with HTTP servers.

We are going to use two python libraries for programming on top of ElasticSearch: `elasticsearch` and `elasticsearch-dsl`. Both provide access to ElasticSearch functionalities hiding and making more programming-friendly the interactions, the second one is more convenient for configurating and searching. Make sure both python libraries are installed to proceed with this session.

We are only going to see the essential elements for developing the session but feel free to learn more.

To interact with ElasticSearch with need a client object of type `Elasticsearch`.

In [336]:
from elasticsearch import Elasticsearch

client = Elasticsearch("http://localhost:9200", timeout=1000)

C:\Users\Carlos Arbonés\AppData\Local\Temp\ipykernel_9268\2120730465.py:3: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  client = Elasticsearch("http://localhost:9200", timeout=1000)


With this client you have a connection for operating with Elastic search. Now we will create an index. There are index operations in each library, but the one in `elasticseach-dsl` is simpler to use.

In [337]:
from elasticsearch_dsl import Index

index = Index('test', using=client)  # if it does not exist, it is created; if it does exist, then it connects

First we will need some text to index, for testing purposes we are going to use the python library `loremipsum`. We will need to install it first if it is not installed already, uncomment the code in next cell if you need to install the library

Now we create some random paragraphs

In [338]:
import lorem

texts = [lorem.paragraph() for _ in range(10)]
print(len(texts))
pprint(texts[0])

10
('Non tempora porro consectetur magnam. Modi sed ut aliquam tempora. Aliquam '
 'eius etincidunt non porro. Neque tempora porro tempora quisquam ipsum '
 'quiquia. Sit ipsum neque aliquam magnam etincidunt etincidunt est. Tempora '
 'tempora tempora porro non non.')


Now we can index the paragraphs in ElasticSearch using the `index` method. The document is passed as a python dictionary with the `document` parameter. The keys of the dictionary will be the fields of the document, in this case we well have only one (`text`) -- here, we use this tag but could use anything we wanted to.

In [339]:
for t in texts:
    client.index(index='test', document={'text': t})
    print(f'Indexing new text: {t[:70]} ...')

Indexing new text: Non tempora porro consectetur magnam. Modi sed ut aliquam tempora. Ali ...
Indexing new text: Non sit modi dolore etincidunt quaerat ipsum neque. Voluptatem dolor i ...
Indexing new text: Quisquam consectetur porro etincidunt. Etincidunt sit amet aliquam. Al ...
Indexing new text: Dolore non quiquia modi amet velit velit. Non numquam non quiquia ut q ...
Indexing new text: Ut magnam sed sed adipisci dolorem. Dolor ut ipsum porro magnam sed qu ...
Indexing new text: Non quiquia etincidunt labore porro. Dolor quaerat magnam ipsum numqua ...
Indexing new text: Amet dolorem quaerat dolorem. Numquam porro sit quiquia eius consectet ...
Indexing new text: Sit modi eius consectetur sit voluptatem modi. Ipsum neque dolor modi  ...
Indexing new text: Quaerat magnam amet quaerat. Amet amet quaerat dolorem amet. Tempora d ...
Indexing new text: Ut porro numquam aliquam sit. Quiquia consectetur porro dolor quisquam ...


In case we want to get all docs in the index, we can do the following:

In [340]:
# get all docs in index 'test'
resp = client.search(index="test", query={"match_all": {}})

# print them
print(f"Got {resp['hits']['total']['value']} hits:")
for hit in resp['hits']['hits']:
    pprint(hit["_source"])

Got 0 hits:


We can also search for documents that contain a given keyword:

In [341]:
from elasticsearch_dsl import Search

# the following search query specifies the field where we want to search
s_obj = Search(using=client, index='test')
sq = s_obj.query('match', text='non')
resp = sq.execute()

print(f'Found {len(resp)} matches.')

for hit in resp:
    print(f'\nID: {hit.meta.id}\nText: {hit.text}')

Found 0 matches.


## 4. Counting words and docs

`Elastic search` helps us to obtain the counts of words in each document. For example, the following code obtains the counts of words of a whole index by adding the counts of words obtained from each document through the functionality of `termvectors`. This function also allows us to get _document counts_ for computing tf-idf weights, by setting the `term_statistics` option to `True`.

In [342]:
from elasticsearch.helpers import scan
from collections import Counter

# Search for all the documents and query the list of (word, frequency) of each one
# Totals are accumulated using a Counter for term frequencies, but not for document freqs.
word_counts = Counter()
doc_counts = Counter()
sc = scan(client, index='test', query={"query" : {"match_all": {}}})
for s in sc:
    tv = client.termvectors(index='test', id=s['_id'], fields=['text'], term_statistics=True, positions=False)
    if 'text' in tv['term_vectors']:   # just in case some document has no field named 'text'
        for t in tv['term_vectors']['text']['terms']:
            word = t
            count = tv['term_vectors']['text']['terms'][t]['term_freq']
            df = tv['term_vectors']['text']['terms'][t]['doc_freq']
            #pprint(tv['term_vectors']['text']['terms'][t])
            word_counts.update({word: count})  # número de veces que aparece la palabra en el documento
            doc_counts[word] = df      # número de documentos en los que aparece la palabra

In [343]:
# show word frequencies
word_counts.most_common()

[]

In [344]:
# show doc freq
doc_counts.most_common()

[]

## 5. Proposed simple exercise

To get more familiar with elasticsearch, we propose that you _generate the Boolean and tf-idf matrices_ for the toy example that we used in class. You will find 7 text documents that contain the toy documents with the materials for this session in the racó. The steps to follow are:

- create an empty index
- open each text document in the `toy-docs` folder provided, read its contents and add it to the index as a new document; your index should contain 7 documents after this
- use the `termvectors` function to obtain term and doc counts, generate Boolean and tf-idf matrices based on these counts
- double check that your results coincide with the numbers in theory slides

In [345]:
index = Index('exercise', using=client)

In [346]:
for i in range(1, 8):
    with open('toy-docs/' + f'd{i}.txt', 'r') as f:
        text = f.read()
        client.index(index='exercise', document={'text': text})

In [347]:
resp = client.search(index='exercise', query={"match_all": {}})
print(f"Got {resp['hits']['total']['value']} hits")

Got 14 hits


In [348]:
import math
D = resp['hits']['total']['value']

boolean_matrix = np.zeros((D, 6), dtype=int)
w_matrix = np.zeros((D, 6))
map = {'five': 0, 'four': 1, 'one': 2, 'six': 3, 'three': 4, 'two': 5}	

word_counts = Counter()
doc_counts = Counter()
sc = scan(client, index='exercise', query={"query" : {"match_all": {}}})

for i, s in enumerate(sc):
    tv = client.termvectors(index='exercise', 
                            id=s['_id'], 
                            fields=['text'], 
                            term_statistics=True, 
                            positions=False)

    if 'text' in tv['term_vectors']:
        # get the word with the highest frequency and its frequency
        max_word = max(tv['term_vectors']['text']['terms'], key=lambda x: tv['term_vectors']['text']['terms'][x]['term_freq'])
        max_freq = tv['term_vectors']['text']['terms'][max_word]['term_freq']
        
        for t in tv['term_vectors']['text']['terms']:
            word = t
            
            count = tv['term_vectors']['text']['terms'][t]['term_freq']
            df = tv['term_vectors']['text']['terms'][t]['doc_freq']
            word_counts.update({word: count})  
            doc_counts[word] = df

            boolean_matrix[i][map[word]] = 1
            w_matrix[i][map[word]] = (count/max_freq) * math.log(D/df, 2)


pprint(boolean_matrix)
print('\n')

np.set_printoptions(precision=2)
pprint(w_matrix)

array([[0, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 1, 1],
       [1, 1, 1, 0, 1, 0],
       [0, 0, 1, 1, 1, 1],
       [0, 1, 0, 1, 1, 0],
       [0, 0, 0, 1, 1, 0],
       [1, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 1, 1],
       [1, 1, 1, 0, 1, 0],
       [0, 0, 1, 1, 1, 1],
       [0, 1, 0, 1, 1, 0],
       [0, 0, 0, 1, 1, 0],
       [1, 1, 0, 0, 0, 0]])


array([[0.  , 0.  , 1.22, 0.  , 0.22, 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.11, 1.81],
       [1.81, 0.41, 0.41, 0.  , 0.07, 0.  ],
       [0.  , 0.  , 0.31, 0.61, 0.06, 1.81],
       [0.  , 1.22, 0.  , 0.41, 0.07, 0.  ],
       [0.  , 0.  , 0.  , 0.81, 0.22, 0.  ],
       [1.81, 1.22, 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 1.22, 0.  , 0.22, 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.11, 1.81],
       [1.81, 0.41, 0.41, 0.  , 0.07, 0.  ],
       [0.  , 0.  , 0.31, 0.61, 0.06, 1.81],
       [0.  , 1.22, 0.  , 0.41, 0.07, 0.  ],
       [0.  , 0.  , 0.  , 0.81, 0.22, 0.  ],
       [1.81, 1.22, 0.  , 0.  , 0.

## 6. Cleanup

Finally, we remove the test index..

In [349]:
index.delete()

ObjectApiResponse({'acknowledged': True})